# Mamba Validation Panel — Sample Walkthrough

Inspect the validation panel used to qualify the Act_of_God Mamba run. This notebook uses `data-sample/mamba_validation_panel/validation_panel_sample.csv` but mirrors the aggregation performed in `data-processed/mamba_ssm/validation_panel_counts.csv`.

In [ ]:
from pathlib import Path
import sys
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('ggplot')

project_root = Path.cwd()
sys.path.append(str(project_root / 'src'))

sample_path = project_root / 'data-sample' / 'mamba_validation_panel' / 'validation_panel_sample.csv'
panel = pd.read_csv(sample_path)
panel.head()


Compare the sample subset against the full validation panel counts stored in `data-processed/mamba_ssm/validation_panel_counts.csv`.

In [ ]:
full_counts = pd.read_csv(project_root / 'data-processed/mamba_ssm/validation_panel_counts.csv')
full_counts.head()

Load the readiness metrics produced during the Act_of_God run (per-group summary).

In [ ]:
panel_eval = pd.read_csv(project_root / 'ops/output/data/mamba_checks/panel_eval/mamba_tiny_uv_best_per_group_summary.csv')
panel_eval[['group_id', 'psnr_mean', 'sam_mean', 'spectra_count']].head()

Display the shipped checkpoint hash so auditors can align the evaluation with the correct weights.

In [ ]:
checksums = (project_root / 'models/mamba_ssm/CHECKSUMS.sha256').read_text().strip().splitlines()
for line in checksums[:5]:
    print(line)

Aggregate spectra counts by treatment and angle to reproduce the readiness-gate checks.

In [ ]:
counts = panel.groupby(['treatment', 'angle']).agg(spectra=('relative_path', 'count'), unique_samples=('sample', 'nunique')).reset_index()
counts


Visualise the distribution to confirm coverage.

In [ ]:
fig, ax = plt.subplots(figsize=(6, 4))
for angle, subset in counts.groupby('angle'):
    ax.bar(subset['treatment'] + '
' + angle, subset['spectra'], label=angle)
ax.set_ylabel('Spectra count')
ax.set_title('Validation panel coverage (sample subset)')
ax.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


### Full Evaluation

- Run `python src/mamba_ssm/scripts/evaluate_validation_panel.py --manifest data/reference/mamba_ssm/validation_panel.csv` to compute the readiness metrics using the shipped checkpoints in `models/mamba_ssm/`.
- The complete aggregated counts are stored in `data-processed/mamba_ssm/validation_panel_counts.csv`.
- See `docs/mamba_ssm/index.md` for a description of validation gates and downstream proxy checks.